In [62]:
from keras.models import Sequential, Model
from keras.layers.embeddings import Embedding
from keras.layers import Input, Activation, Dense, Permute, Dropout, add, dot, concatenate
from keras.layers import LSTM
from keras.utils.data_utils import get_file
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from functools import reduce
import pickle
import tarfile
import numpy as np
import re
import os
import time


def tokenize(sent):
    '''Return the tokens of a sentence including punctuation.
    >>> tokenize('Bob dropped the apple. Where is the apple?')
    ['Bob', 'dropped', 'the', 'apple', '.', 'Where', 'is', 'the', 'apple', '?']
    '''
    return [x.strip() for x in re.split('(\W+)', sent) if x.strip()]


def parse_stories(lines, only_supporting=False):
    '''Parse stories provided in the bAbi tasks format
    If only_supporting is true, only the sentences
    that support the answer are kept.
    '''
    data = []
    story = []
    for line in lines:
        line = line.decode('utf-8').strip()
        nid, line = line.split(' ', 1)
        nid = int(nid)
        if nid == 1:
            story = []
        if '\t' in line:
            q, a, supporting = line.split('\t')
            q = tokenize(q)
            substory = None
            if only_supporting:
                # Only select the related substory
                supporting = map(int, supporting.split())
                substory = [story[i - 1] for i in supporting]
            else:
                # Provide all the substories
                substory = [x for x in story if x]
            data.append((substory, q, a))
            story.append('')
        else:
            sent = tokenize(line)
            story.append(sent)
    return data


def get_stories(f, only_supporting=False, max_length=None):
    '''Given a file name, read the file,
    retrieve the stories,
    and then convert the sentences into a single story.
    If max_length is supplied,
    any stories longer than max_length tokens will be discarded.
    '''
    data = parse_stories(f.readlines(), only_supporting=only_supporting)
    flatten = lambda data: reduce(lambda x, y: x + y, data)
    data = [(flatten(story), q, answer) for story, q, answer in data if not max_length or len(flatten(story)) < max_length]
    return data


def vectorize_stories(data):
    inputs, queries, answers = [], [], []
    for story, query, answer in data:
        inputs.append([word_idx[w] for w in story])
        queries.append([word_idx[w] for w in query])
        answers.append(word_idx[answer])
    return (pad_sequences(inputs, maxlen=story_maxlen),
            pad_sequences(queries, maxlen=query_maxlen),
            np.array(answers))


In [68]:
try:
    path = 'C:/Users/310267647/.keras/datasets/babi-tasks-v1-2.tar8.gz'
except:
  #  print('Error downloading dataset, please download it manually:\n'
 #         '$ wget http://www.thespermwhale.com/jaseweston/babi/tasks_1-20_v1-2.tar.gz\n'
  #        '$ mv tasks_1-20_v1-2.tar.gz ~/.keras/datasets/babi-tasks-v1-2.tar.gz')
    raise
tar = tarfile.open(path)

challenges = {
    # QA1 with 10,000 samples
    'single_supporting_fact_10k': 'tasks_1-20_v1-2/en-10k/qa1_single-supporting-fact_{}.txt',
    # QA2 with 10,000 samples
    'two_supporting_facts_10k': 'tasks_1-20_v1-2/en-10k/qa2_two-supporting-facts_{}.txt',
}
challenge_type = 'single_supporting_fact_10k'
challenge = challenges[challenge_type]

print('Extracting stories for the challenge:', challenge_type)
train_stories = get_stories(tar.extractfile(challenge.format('train')))
test_stories = get_stories(tar.extractfile(challenge.format('test')))

Extracting stories for the challenge: single_supporting_fact_10k


In [69]:

# See what the data looks like# See w 

for i in range(10):
    print("Story: {}".format(' '.join(train_stories[i][0])))
    print("Query: {}".format(' '.join(train_stories[i][1])))
    print("Answer: {}".format(train_stories[i][2]))
    print("---")

Story: Task today is tasktoday . Task for yesterday was taskyesterday .
Query: What is task for today ?
Answer: tasktoday
---
Story: Task today is tasktoday . Task for yesterday was taskyesterday . Task for tomorrow is tasktomorrow . Task for today is tasktoday .
Query: What is task for today ?
Answer: tasktoday
---
Story: Task today is tasktoday . Task for yesterday was taskyesterday . Task for tomorrow is tasktomorrow . Task for today is tasktoday . Task for yesterday was taskyesterday . Task for tomorrow is tasktomorrow .
Query: What is task for today ?
Answer: tasktoday
---
Story: Task today is tasktoday . Task for yesterday was taskyesterday . Task for tomorrow is tasktomorrow . Task for today is tasktoday . Task for yesterday was taskyesterday . Task for tomorrow is tasktomorrow . Task for tomorrow is tasktomorrow . Task for today is tasktoday .
Query: What is task for today ?
Answer: tasktoday
---
Story: Task today is tasktoday . Task for yesterday was taskyesterday . Task for t

In [70]:
vocab = set()
for story, q, answer in train_stories + test_stories:
    vocab |= set(story + q + [answer])
vocab = sorted(vocab)

# Reserve 0 for masking via pad_sequences
vocab_size = len(vocab) + 1
story_maxlen = max(map(len, (x for x, _, _ in train_stories + test_stories)))
query_maxlen = max(map(len, (x for _, x, _ in train_stories + test_stories)))

print('-')
print('Vocab size:', vocab_size, 'unique words')
print('Story max length:', story_maxlen, 'words')
print('Query max length:', query_maxlen, 'words')
print('Number of training stories:', len(train_stories))
print('Number of test stories:', len(test_stories))
print('-')
print('Here\'s what a "story" tuple looks like (input, query, answer):')
print(train_stories[0])
print('-')


for s in list(enumerate(vocab)):
    print(s)

-
Vocab size: 15 unique words
Story max length: 60 words
Query max length: 6 words
Number of training stories: 10
Number of test stories: 10
-
Here's what a "story" tuple looks like (input, query, answer):
(['Task', 'today', 'is', 'tasktoday', '.', 'Task', 'for', 'yesterday', 'was', 'taskyesterday', '.'], ['What', 'is', 'task', 'for', 'today', '?'], 'tasktoday')
-
(0, '.')
(1, '?')
(2, 'Task')
(3, 'What')
(4, 'for')
(5, 'is')
(6, 'task')
(7, 'tasktoday')
(8, 'tasktomorrow')
(9, 'taskyesterday')
(10, 'today')
(11, 'tomorrow')
(12, 'was')
(13, 'yesterday')


In [71]:
print('Vectorizing the word sequences...')
word_idx = dict((c, i + 1) for i, c in enumerate(vocab))
inputs_train, queries_train, answers_train = vectorize_stories(train_stories)
inputs_test, queries_test, answers_test = vectorize_stories(test_stories)

print('-')
print('inputs: integer tensor of shape (samples, max_length)')
print('inputs_train shape:', inputs_train.shape)
print('inputs_test shape:', inputs_test.shape)
print('-')
print('queries: integer tensor of shape (samples, max_length)')
print('queries_train shape:', queries_train.shape)
print('queries_test shape:', queries_test.shape)
print('-')
print('answers: binary (1 or 0) tensor of shape (samples, vocab_size)')
print('answers_train shape:', answers_train.shape)
print('answers_test shape:', answers_test.shape)
print('-')

Vectorizing the word sequences...
-
inputs: integer tensor of shape (samples, max_length)
inputs_train shape: (10, 60)
inputs_test shape: (10, 60)
-
queries: integer tensor of shape (samples, max_length)
queries_train shape: (10, 6)
queries_test shape: (10, 6)
-
answers: binary (1 or 0) tensor of shape (samples, vocab_size)
answers_train shape: (10,)
answers_test shape: (10,)
-


In [72]:

# See individual training element.# See i 

print("Story (x): {}".format(inputs_train[0]))
print("Question (x): {}".format(queries_train[0]))
print("Answer: {}".format(answers_train[0]))

Story (x): [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  3 11  6  8  1  3  5 14 13 10  1]
Question (x): [ 4  6  7  5 11  2]
Answer: 8


In [73]:
print('Compiling...')

# placeholders
input_sequence = Input((story_maxlen,))
question = Input((query_maxlen,))

# encoders
# embed the input sequence into a sequence of vectors
input_encoder_m = Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,
                              output_dim=64))
input_encoder_m.add(Dropout(0.3))
# output: (samples, story_maxlen, embedding_dim)

# embed the input into a sequence of vectors of size query_maxlen
input_encoder_c = Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,
                              output_dim=query_maxlen))
input_encoder_c.add(Dropout(0.3))
# output: (samples, story_maxlen, query_maxlen)

# embed the question into a sequence of vectors
question_encoder = Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,
                               output_dim=64,
                               input_length=query_maxlen))
question_encoder.add(Dropout(0.3))
# output: (samples, query_maxlen, embedding_dim)

# encode input sequence and questions (which are indices)
# to sequences of dense vectors
input_encoded_m = input_encoder_m(input_sequence)
input_encoded_c = input_encoder_c(input_sequence)
question_encoded = question_encoder(question)

# compute a 'match' between the first input vector sequence
# and the question vector sequence
# shape: `(samples, story_maxlen, query_maxlen)`
match = dot([input_encoded_m, question_encoded], axes=(2, 2))
match = Activation('softmax')(match)

# add the match matrix with the second input vector sequence
response = add([match, input_encoded_c])  # (samples, story_maxlen, query_maxlen)
response = Permute((2, 1))(response)  # (samples, query_maxlen, story_maxlen)

# concatenate the match matrix with the question vector sequence
answer = concatenate([response, question_encoded])

# the original paper uses a matrix multiplication for this reduction step.
# we choose to use a RNN instead.
answer = LSTM(32)(answer)  # (samples, 32)

# one regularization layer -- more would probably be needed.
answer = Dropout(0.3)(answer)
answer = Dense(vocab_size)(answer)  # (samples, vocab_size)
# we output a probability distribution over the vocabulary
answer = Activation('softmax')(answer)

# build the final model
model = Model([input_sequence, question], answer)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
print("Done.")

Compiling...
Done.


In [104]:
start_time = time.time()
# train
model.fit([inputs_train, queries_train], answers_train,
          batch_size=32,
          epochs=120,
          validation_data=([inputs_test, queries_test], answers_test))

# save
save_path = "C:/Users/310267647/.keras/datasets/"
# save entire network to HDF5 (save everything, suggested)
model.save(os.path.join(save_path,"chatbot.h5"))
# save the vocab too, indexes must be the same
pickle.dump( vocab, open( os.path.join(save_path,"vocab.pkl"), "wb" ) )

elapsed_time = time.time() - start_time
#print("Elapsed time: {}".format(hms_string(elapsed_time)))


Train on 10 samples, validate on 10 samples
Epoch 1/120
10/10 [==============================] - 0s - loss: 0.9063 - acc: 0.6000 - val_loss: 0.8517 - val_acc: 0.5000
Epoch 2/120
10/10 [==============================] - 0s - loss: 0.7695 - acc: 0.8000 - val_loss: 0.8333 - val_acc: 0.5000
Epoch 3/120
10/10 [==============================] - 0s - loss: 0.8672 - acc: 0.7000 - val_loss: 0.8180 - val_acc: 0.5000
Epoch 4/120
10/10 [==============================] - 0s - loss: 0.8827 - acc: 0.4000 - val_loss: 0.8111 - val_acc: 0.5000
Epoch 5/120
10/10 [==============================] - 0s - loss: 0.8824 - acc: 0.4000 - val_loss: 0.8003 - val_acc: 0.5000
Epoch 6/120
10/10 [==============================] - 0s - loss: 0.7524 - acc: 0.6000 - val_loss: 0.7871 - val_acc: 0.5000
Epoch 7/120
10/10 [==============================] - 0s - loss: 1.1520 - acc: 0.2000 - val_loss: 0.7803 - val_acc: 0.5000
Epoch 8/120
10/10 [==============================] - 0s - loss: 0.6867 - acc: 0.7000 - val_loss: 0.766

10/10 [==============================] - 0s - loss: 0.2022 - acc: 1.0000 - val_loss: 0.1146 - val_acc: 1.0000
Epoch 68/120
10/10 [==============================] - 0s - loss: 0.1148 - acc: 1.0000 - val_loss: 0.1090 - val_acc: 1.0000
Epoch 69/120
10/10 [==============================] - 0s - loss: 0.1535 - acc: 1.0000 - val_loss: 0.1070 - val_acc: 1.0000
Epoch 70/120
10/10 [==============================] - 0s - loss: 0.1502 - acc: 1.0000 - val_loss: 0.1062 - val_acc: 1.0000
Epoch 71/120
10/10 [==============================] - 0s - loss: 0.0958 - acc: 1.0000 - val_loss: 0.0935 - val_acc: 1.0000
Epoch 72/120
10/10 [==============================] - 0s - loss: 0.0872 - acc: 1.0000 - val_loss: 0.0875 - val_acc: 1.0000
Epoch 73/120
10/10 [==============================] - 0s - loss: 0.0957 - acc: 1.0000 - val_loss: 0.0832 - val_acc: 1.0000
Epoch 74/120
10/10 [==============================] - 0s - loss: 0.0988 - acc: 1.0000 - val_loss: 0.0786 - val_acc: 1.0000
Epoch 75/120
10/10 [=========

In [105]:

# Load the model, if it exists, load vocab too
save_path = "C:/Users/310267647/.keras/datasets/"
model = load_model(os.path.join(save_path,"chatbot.h5"))
vocab = pickle.load( open( os.path.join(save_path,"vocab.pkl"), "rb" ) )

In [106]:
pred = model.predict([inputs_test, queries_test])
# See what the predictions look like, they are just probabilities of each class.
print(pred)

[[1.46445323e-04 2.29949830e-04 1.37360534e-04 2.65870505e-04
  3.72650393e-04 1.59459276e-04 7.08726075e-05 1.74221364e-04
  9.83692050e-01 1.03552057e-03 1.21412994e-02 4.64760466e-04
  5.36303967e-04 3.34551878e-04 2.38698980e-04]
 [1.47957267e-04 2.36868596e-04 1.38159754e-04 2.74190475e-04
  3.72426701e-04 1.58981915e-04 6.92500180e-05 1.81049007e-04
  9.84245121e-01 1.04032014e-03 1.15229189e-02 4.75112174e-04
  5.45805611e-04 3.49255773e-04 2.42442766e-04]
 [1.50757987e-04 2.38692272e-04 1.39892640e-04 2.74852879e-04
  3.78800265e-04 1.60344571e-04 7.09761298e-05 1.81664931e-04
  9.84003127e-01 1.06529752e-03 1.17036225e-02 4.79606184e-04
  5.53729304e-04 3.54458723e-04 2.43959890e-04]
 [1.55803282e-04 2.47488177e-04 1.43719677e-04 2.77600542e-04
  3.82754981e-04 1.71333610e-04 7.15834030e-05 1.86043675e-04
  9.84421253e-01 1.08853995e-03 1.11796940e-02 4.78396309e-04
  5.69860218e-04 3.73542396e-04 2.52296770e-04]
 [1.57840157e-04 2.43616494e-04 1.43025711e-04 2.76496547e-04
  

In [107]:
# Use argmax to turn those into actual predictions.  The class (word) with the highest
# probability is the answer.

pred = np.argmax(pred,axis=1)
print(pred)

[ 8  8  8  8  8 10 10 10 10 10]


In [108]:
score = metrics.accuracy_score(answers_test, pred)
print("Final accuracy: {}".format(score))

Final accuracy: 1.0


In [110]:
print("Remember, I only know these words: {}".format(vocab))
print()
story = "Task today is tasktoday. Task for tomorrow is tasktomorrow. Task for yesterday was taskyesterday."
query = "What is task for yesterday?"

adhoc_stories = (tokenize(story), tokenize(query), '?')

adhoc_train, adhoc_query, adhoc_answer = vectorize_stories([adhoc_stories])

pred = model.predict([adhoc_train, adhoc_query])
print(pred[0])
pred = np.argmax(pred,axis=1)
print("Answer: {}({})".format(vocab[pred[0]-1],pred))

Remember, I only know these words: ['.', '?', 'Task', 'What', 'for', 'is', 'task', 'tasktoday', 'tasktomorrow', 'taskyesterday', 'today', 'tomorrow', 'was', 'yesterday']

[3.1849337e-05 1.7253027e-04 1.9919404e-05 1.5431790e-04 6.3069027e-05
 1.1277236e-04 1.7380287e-04 4.7067351e-05 5.5776616e-03 7.4987591e-05
 9.9321401e-01 1.5879613e-04 7.4054755e-05 6.6307461e-05 5.8981997e-05]
Answer: taskyesterday([10])
